In [35]:
%pip install torch
%pip install torchvision


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [37]:
#define the neural network architecture

class SimpleNN(nn.Module):
    '''
    A simple feed-forward neural network for image classification
    '''

    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(SimpleNN, self).__init__()
        self.flatten =nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, num_classes),
        )

    def forward(self, x):
        '''
        Defines the forward pass through the network
        '''
        x = self.flatten(x)
        logits = self.network(x)
        return logits

In [38]:
#Set up Hyperparameters and Device
INPUT_SIZE = 784
HIDDEN_SIZE1 = 128
HIDDEN_SIZE2 = 64
NUM_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [39]:
#Load and Prepare the dataset

#Define a tansform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

#Download and load the datasets
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transform
)

#Create DataLoaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True #to remove autocorrelation
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False 
)


In [40]:
#Initialize the Model, Loss Function and Optimizer
model = SimpleNN(INPUT_SIZE, HIDDEN_SIZE1, HIDDEN_SIZE2, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [41]:
#Train the Network
print("Training the network...")
for epoch in range(NUM_EPOCHS):
    for i, (images,labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

print("Training complete!")

Training the network...
Epoch [1/5], Step [100/938], Loss: 0.3882
Epoch [1/5], Step [200/938], Loss: 0.3591
Epoch [1/5], Step [300/938], Loss: 0.2839
Epoch [1/5], Step [400/938], Loss: 0.3312
Epoch [1/5], Step [500/938], Loss: 0.2854
Epoch [1/5], Step [600/938], Loss: 0.3094
Epoch [1/5], Step [700/938], Loss: 0.2869
Epoch [1/5], Step [800/938], Loss: 0.3553
Epoch [1/5], Step [900/938], Loss: 0.2682
Epoch [2/5], Step [100/938], Loss: 0.0791
Epoch [2/5], Step [200/938], Loss: 0.3039
Epoch [2/5], Step [300/938], Loss: 0.1536
Epoch [2/5], Step [400/938], Loss: 0.1893
Epoch [2/5], Step [500/938], Loss: 0.1440
Epoch [2/5], Step [600/938], Loss: 0.0784
Epoch [2/5], Step [700/938], Loss: 0.1608
Epoch [2/5], Step [800/938], Loss: 0.2113
Epoch [2/5], Step [900/938], Loss: 0.1320
Epoch [3/5], Step [100/938], Loss: 0.1675
Epoch [3/5], Step [200/938], Loss: 0.2380
Epoch [3/5], Step [300/938], Loss: 0.0603
Epoch [3/5], Step [400/938], Loss: 0.1636
Epoch [3/5], Step [500/938], Loss: 0.0726
Epoch [3/5

In [42]:
print("Evaluating the model on test data...")
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct =0
    total =0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
    print(f"Accuracy of the network on the 10000 test images: {accuracy:.2f}%")


Evaluating the model on test data...
Accuracy of the network on the 10000 test images: 96.15%
